# Encryption With private and public keys
This was first introduced to the Milvus_Service directory but after a discussion it was planned to introduce it later when actual sensitive information is available as the only thing saved for now is the users bots information and nothing from the user itself.

The choice of implementing this is that based on that a part of the encryption can be made available to other instances such as multiple bots that can save to a central database. Although they know a part of the encryption process a part is still save from immediately unraveling the encryption placed on it.

## The Library
- os: is used to connect to the env file
- dotenv: used to read the env file stored on disk
- cryptography: used to interact with the generated keys
- base64: to enable it to save it to the database

In [2]:
import os
from dotenv import load_dotenv
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.backends import default_backend
import base64

If the following is true then it can read a env file in the directory.
When false then an env is not found.

In [3]:
load_dotenv()

True

## Methods

The encrypt_with_public_key method needs to show how you can encrypt your string, with the public key.
I first serialize the key and then encrypt the message (string).



In [4]:
def encrypt_with_public_key(public_key_str, message):

    public_key = serialization.load_pem_public_key(
        public_key_str.encode(),
        backend=default_backend()
    )

    encrypted = public_key.encrypt(
        message,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )

    return encrypted

The method decrypt_with_private_key: this is a method that needs to decrypt a decoded base64 string to a normal unicode string.  
First it gets the privatekey from the env file and because of how the private key is structured we change it from one line to multiline.  
Then we check the private key on errors and then when there is no errors we try to decrypt the text given.

In [5]:
def decrypt_with_private_key(ciphertext):

    private_key_str = os.environ.get("PRIVATE_KEY")    
    private_key_str = private_key_str.replace("\\n", "\n")
    private_key_str = private_key_str.replace("\\", "")
    # Check if the private key string is None or empty
    if not private_key_str:
        raise ValueError("The PRIVATE_KEY environment variable is not set or is empty.")

    try:
        private_key = serialization.load_pem_private_key(
            private_key_str.encode(),
            password=None,
            backend=default_backend()
        )
    except ValueError as e:
        raise ValueError(
            "Could not load the private key. The key may be encrypted, in which case you need to provide the "
            "password.") from e

    decrypted = private_key.decrypt(
        ciphertext,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )

    return decrypted.decode()

## Encryption


Now for some random values which I made up. This should be pseudo-sensitive information as name is definitely sensitive but the description itself is questionable.

In [6]:
name = "Casey"
description = "A normal person"

We first get the public key from the environment variable.  
We then start with encrypting it with the public key and then we encode it to base64.

In [7]:
public_key = os.environ.get("PUBLIC_KEY")  # replace with your public key path
encrypted_name = encrypt_with_public_key(public_key, name.encode())
encrypted_description = encrypt_with_public_key(public_key, description.encode())

# Convert encrypted bytes to Base64 strings
base64_name = base64.b64encode(encrypted_name).decode()
base64_description = base64.b64encode(encrypted_description).decode()

## Decoding

we start with decoding it to base64.
Then we get the decryption with the private key and now we have "normal" values

In [8]:
encrypted_name = base64.b64decode(base64_name)
encrypted_description = base64.b64decode(base64_description)

# get decrypted values
decrypted_name = decrypt_with_private_key(encrypted_name)
decrypted_description = decrypt_with_private_key(encrypted_description)


In [9]:
print(decrypted_name)
print(decrypted_description)

Casey
A normal person


## Introspective and testing

Now with some stranger characters. I found some non-unicode characters combined with unicode characters and they also work the same.
The first 3 characters are non-unicode and the others are unicode.

In [10]:
name="ÿあΩᶁ⠃⟫강𐀐"
print(name)
public_key = os.environ.get("PUBLIC_KEY")  # replace with your public key path
encrypted_name = encrypt_with_public_key(public_key, name.encode())
print(encrypted_name)
base64_name = base64.b64encode(encrypted_name).decode()
print(base64_name)
encrypted_name = base64.b64decode(base64_name)
print(encrypted_name)
decrypted_name = decrypt_with_private_key(encrypted_name)
print(decrypted_name)

ÿあΩᶁ⠃⟫강𐀐
b'\x8dL\x85\xe8!\xbc\xf3M\xdcW%\x0e\x98\xfa7.\x0b"\xa3\xf7K\x17\x1b\x0e\x1c\xf6r$\xc9\xa2&\xc6>\xbdz\xd5\xc9`S\x1b\xe0\xb8\xfc\xfcv>\xff9\xe9\x0bJ{X\x1d\xcc\xcer\xcc\xd2u\xa8\xbb\xf3Y\xd1\xd9\xff\x10\x19k\xa2\x8cU\xd5\xe1\x1c\x97\xbb\xe4?\\\x16U3p#\xb8\xcfX\x87\x14\x8c\xcdo H\xc2\xb6\x9b\xa7=i\xca\xf8\xe1\x073\x7f\xf3\xf4W\x8d\x96\xe7C\x87\x84\x08\xe5|\x9d\x0f{\x98\xa9"\x92\x10k\x8cci\x08u\xe6\x82\xbc\xc11}\x0f\xd0\xaa:\x08\x82\xa13\x91W\xeftjT\xcd)\xde=\xa7\xe8\x83(D\xa0\xc9I\xc8f\xb2p\xfe\xc4P\xd4\x87\xbc\x14\x8d\x87l\xfc\x05\x0f\xfbg\x8f\xc5\x16\xf9\xc4\xc2\x1bf\xa0\xf4Z+\xe9\xbf\'f\xe5\xc5x\xce\n\xa3\xdc$\xa6A\xcb\xebz)5\xf4C\xc1\xa6\x16\x8dc\xf4m\xb1\xf0\xae\xb9&4\'5\xe0oq\x97\x15U\x8b76\xe8\x1c\x92\xc2\xa8l\x88x\xa0\x9aR\xbf\xb2\x10'
jUyF6CG8803cVyUOmPo3Lgsio/dLFxsOHPZyJMmiJsY+vXrVyWBTG+C4/Px2Pv856QtKe1gdzM5yzNJ1qLvzWdHZ/xAZa6KMVdXhHJe75D9cFlUzcCO4z1iHFIzNbyBIwrabpz1pyvjhBzN/8/RXjZbnQ4eECOV8nQ97mKkikhBrjGNpCHXmgrzBMX0P0Ko6CIKhM5FX73RqVM0p3j2n6IMoRKDJSchmsnD+xFDUh7wUjYds/

I can also try to perform double encryption with having it do encrypt, base64, encrypt, base64 and let's see what changes.

In [11]:
name="A simple message"
print(name)
public_key = os.environ.get("PUBLIC_KEY")  # replace with your public key path
# First encryption
encrypted_name = encrypt_with_public_key(public_key, name.encode())
print(encrypted_name)
base64_name = base64.b64encode(encrypted_name).decode()
print(base64_name)
# Second encryption
encrypted_name = encrypt_with_public_key(public_key, base64_name.encode())
print(encrypted_name)
base64_name = base64.b64encode(encrypted_name).decode()
print(base64_name)

# First decrypt
encrypted_name = base64.b64decode(base64_name)
print(encrypted_name)
decrypted_name = decrypt_with_private_key(encrypted_name)
print(decrypted_name)
# Second decrypt
encrypted_name = base64.b64decode(base64_name)
print(encrypted_name)
decrypted_name = decrypt_with_private_key(encrypted_name)
print(decrypted_name)

A simple message
b'1N\xc2\xb9i\xc5\xc9\x8aP\xe7|\x0e\xf6\xec\xda\xf5B\x86\xe1\x91>\xad\xa7\x88\xe4>\xaf\xba\xfcJ"\xa4\x1c\xe3\x82~6+H\x19\x8b\xcf_\xfc\xecA\x89E\xc9\x8a\xbf\x7f;\xf6n\x0f/\xc6\xbbO?+\xd6\xbel"\x10\x1e\x8fk\xeaK7\xb4\r\x91Z\rJ\xd4\x83Z\xf6\x9e\x1d\xf2X\xb9\x95B\x10w\x05a\xd9\x7f\x02&\xf6\xda\xd7{we\x93\xe4\xf9\xe5\xf5\x81\x12\xecW\xbe\xcf\xd1\x8b\xe1\xe5?\x19\x0f\xc1\xe8\xac>G\x0e\x91\x82\xaa\x0cE\xbf&\xec\xb0\xb0\xacb\xb2\xdf\x9b;\xd0\x84v\x9a\xc4\xc8\xee\xe1\xde\x88 P\x97\x97\x9c\x08!\xd2\xe6?Q\xe2\xd2\xd5\x9a\xa3L\x8b\xfc\x8e\x08Oo\xe4\xb7P*\xad\x83\\T\x93\xc5c\xd1\xff\xa8$kl*\x84MP\xa1\x1c\xa2\xc2\x17\xc2S!\xd7\x0c;l3c\xca\x94;\xfd`\xac\xdc\xfa6\xe1\x97V\xe0N\x8d\x80\xc1\xf9\xc8=\xd7\x93\xfd\xb98l\xb5\\\x91\xae7h\x12\xec+_mC\x85N8)\xc8\xa3'
MU7CuWnFyYpQ53wO9uza9UKG4ZE+raeI5D6vuvxKIqQc44J+NitIGYvPX/zsQYlFyYq/fzv2bg8vxrtPPyvWvmwiEB6Pa+pLN7QNkVoNStSDWvaeHfJYuZVCEHcFYdl/Aib22td7d2WT5Pnl9YES7Fe+z9GL4eU/GQ/B6Kw+Rw6RgqoMRb8m7LCwrGKy35s70IR2msTI7uHeiCBQl5ecCCHS5j9R4tLVmqNMi/

ValueError: Encryption failed

As you can see it didn't work. Now for why let's see if we copy the base64 if we can see if it does encrypt.
I start with having it be max 255 characters but this didn't work which I then reduced by 10 for each error, which I then come to 190 which I added first plus 1 (191) and this gave an error. So possibly the cause is a character

In [12]:
name="TmdkSHQrg0+bSfSYI3F66iVj/ckPsu+3VUOuOTMmFwKUYDJzS933HToTMUC207afgu+2j+aPhCnrZcHwMEc2/lMqxZ+GMHDeYutsy6VsTFobckaz7YftHhuLS4IPtS8tJZEt/4G3SLIShrSafsZmgd9tz7MH3CSng9hadHIURiPP57vMi3ons63MsOwdV6Tj92G7w49lTxc2lM43d6Us8aMqfpZrbNGfioD00UT4xldjWfwX1+oMzbLs2+0JeYzg6ThaFLKJCf/E3C3Wj1R04uOTu0LT+sfQizfK2uRrF/bbHeLvGdS+DIQqHs5dN944MwRjDncLl+MsQgWvIB04YQ=="
print(len(name))
name = name[0:190]
print(len(name))

344
190


In [13]:
encrypted_name = encrypt_with_public_key(public_key, name.encode())
print(encrypted_name)
base64_name = base64.b64encode(encrypted_name).decode()
print(base64_name)

b'\x83*\x11W\x04\x12\x9e^I\xbe\x89\xfa\xe7Y\x93\x89\xd9\xb9\x9a\xa0\xc3V\xf5\xa7S\xfb=\xb3\x85w\xe7\xaa\x17\x0b\x0e\xdb\x83\xd8\xa8D\xb3+\xb0\xd0\xf0\xc9\xc9s\xa6\xdb\xc9=\xb0)\x92\xf4\x9bL\xa9\x8d\xd4\xc4\x96\xb9$\xd8\xebI\xa6\x8e\x1eT\xdf$z3.\xabv\xe0uH\x8f\xc1\xd3\xcf\x00\xa0\xab=\t\xca\x1d\xc8\x8c\xf8\xa3\xfejW\\\x04Pf@\xcc|\x1e\xaf\x9e\xb1f\xa9\x17\x1e6)\xb8\xd9\xfc\xc8;\xfa\x96\xd1\xa06\t\x93\xe4\x7f\xb22\xa3\t)9\xbd\xe8\xe3\x1e\xb6\xa4\xce\xf1p\xbb\xb8\xb0.\xd7\xd03\xa2\xef\x17\x92\x04\xd0\xcc\x80\xba\xd3 &\x9c\r\xfa\xc4C\x1aj8\xaf\xbesYi\xed\xef+\xf5%\xd4\x96\x17\xa7~\x05\xa3\x80\x19,\x81\xe2c\xe4\xff\xa6]0\xc1\xb9\xa3\x83\xcb\xbb\xfe\x8f\xa6\t\xcd\x1fV?\x16\x16\xdc7\xb5\xf0\xe53z2^\xf8p~\xcdP\x04\x99Ojz\xdd\x16f\xf3\xa6\xc2;\xa86\xad\x02[\x033\xf0}\xfa\xe6\xb7\xeb'
gyoRVwQSnl5Jvon651mTidm5mqDDVvWnU/s9s4V356oXCw7bg9ioRLMrsNDwyclzptvJPbApkvSbTKmN1MSWuSTY60mmjh5U3yR6My6rduB1SI/B088AoKs9CcodyIz4o/5qV1wEUGZAzHwer56xZqkXHjYpuNn8yDv6ltGgNgmT5H+yMqMJKTm96OMetqTO8XC7uLAu19Azou8XkgTQzIC

So if look for the 191 charter I can find the cause. So I split it and there was no character that was the reason but the length.

In [14]:
name="TmdkSHQrg0+bSfSYI3F66iVj/ckPsu+3VUOuOTMmFwKUYDJzS933HToTMUC207afgu+2j+aPhCnrZcHwMEc2/lMqxZ+GMHDeYutsy6VsTFobckaz7YftHhuLS4IPtS8tJZEt/4G3SLIShrSafsZmgd9tz7MH3CSng9hadHIURiPP57vMi3ons63MsOwdV6Tj92G7w49lTxc2lM43d6Us8aMqfpZrbNGfioD00UT4xldjWfwX1+oMzbLs2+0JeYzg6ThaFLKJCf/E3C3Wj1R04uOTu0LT+sfQizfK2uRrF/bbHeLvGdS+DIQqHs5dN944MwRjDncLl+MsQgWvIB04YQ=="
name = name[189:]

In [15]:
encrypted_name = encrypt_with_public_key(public_key, name.encode())
print(encrypted_name)
base64_name = base64.b64encode(encrypted_name).decode()
print(base64_name)

b'\xa1\x8a\x89{Lk\'\xcd\xd9\xc722+\xe1*\xfd\xd9P\xe4\x83\xb8\xd3\x1f\xdf\x07]\xb0D\xc3\xb3\x96Y"\xd8/\\\x13\xaf\x9bQ\xd8\xe8-\xf8\xff<\xccN\x8d4tI\xab\x93\x01\xb5\xb1\xc0\xb1\xf19\x87D\\\x9e\xe22A\x0f\xacj\xca\x99\x8f\xaf\xfd\x9fw\x89\nV\x9f\xe2\x12\xa7\xac@\xe7YA-\xf6f\x04\xf8\xd1\xaay\x90\r\xe2\xd1\xd7\x95\x1d\x19\xc4\xbe!H\xf6%)\x1eLD+\x1c\x8d\x7f\xba\xc8\xbd<]\x17\xe9\x9fN]\x81\x07\x8f\xec\x9c\xcb\xbc\xf9\xb3\xa7\x19\xb6e\xcc\xfb\r\xe4\xac\x93\x17\xd1\xe7HA\x14\xdf\xa4\xff\x10#\x15\xb5\xb4y\\2\x83_\x0c\xe2\x1e\x84|\t\x1e-!\x9f]\xca\x9e\xcd\xf2\xd8|\xf5\xae\x90\xfb\xdf\xc8tt\xa5e^\xb1\xadf\xfd\x1a\x07)\xceiY\x07\xb1\x01C\xfbK\x0b*\x83\xee\x18\x02\xday\xa1\xfe\x89\xb9\xf7\xbb\x1d\xb8\x8fe\x03\xe4\xc6u\xcb\xfe\xbaE\x8cC\xb8\x8b\\\xdf\xa6\x19\xe9\xf9\x04rb\xed\xf8\x07[O'
oYqJe0xrJ83ZxzIyK+Eq/dlQ5IO40x/fB12wRMOzllki2C9cE6+bUdjoLfj/PMxOjTR0SauTAbWxwLHxOYdEXJ7iMkEPrGrKmY+v/Z93iQpWn+ISp6xA51lBLfZmBPjRqnmQDeLR15UdGcS+IUj2JSkeTEQrHI1/usi9PF0X6Z9OXYEHj+ycy7z5s6cZtmXM+w3krJMX0edIQRTfpP8QIxW1tH

So let's try to combine it. I first try to encrypt the message. Then I will split it into 2 and encrypt that. Set it to base64 and combine it.
I also will try to decrypt it as well.

In [18]:
name="A simple message"
print(name)
public_key = os.environ.get("PUBLIC_KEY")  # replace with your public key path
# First encryption
encrypted_name = encrypt_with_public_key(public_key, name.encode())
print(encrypted_name)
base64_name = base64.b64encode(encrypted_name).decode()
print(base64_name)
print(len(base64_name))

# Splitting it
encrypt_name_part1=base64_name[:190]
encrypt_name_part2=base64_name[190:]
print(encrypt_name_part1)
print(encrypt_name_part2)

# Second encryption first part
encrypted_name_1 = encrypt_with_public_key(public_key, encrypt_name_part1.encode())
print(encrypted_name_1)
base64_name_1 = base64.b64encode(encrypted_name_1).decode()
print(base64_name_1)

# Second encryption second part
encrypted_name_2 = encrypt_with_public_key(public_key, encrypt_name_part2.encode())
print(encrypted_name_2)
base64_name_2 = base64.b64encode(encrypted_name_2).decode()
print(base64_name_2)

# Combine it
second_encrypted_base64_name= base64_name_1+base64_name_2
print(second_encrypted_base64_name)

A simple message
b"\x0e\x9a\x83\xd4?\xae\x04\xa47\x1c\x0c\xef\xfa\\u\xfc\xb2w\xe7\xe3\x19\xa6cj\xd4\xfc\xed\xde\xc3\x87\x03\xa9\x94'p\xa9\xb5\xb0\x132u\x94$\xfc\xc9\xf3\x81\xcfV)^\x18\xec\x81\x13*\xc8\xf0\xfa\x8c\x9bs\xbdnu7\xc9qSI\x15a\xbb\x89\xce\xd1\x1b\x9e\xda|\x82\xcf\xed\x9aj\x88\xd3\xce\xc8U\xf7\xd5\x10 \xd4\x1a\xb4\x82\xafL\xd1\xf9\xf9\xfd\xc1\xda\xac\xe9\xb5\x12?\xdb\x0b7\xea\xe7\x995\x14\x8b\x97\x91r\xc9\xa0\x02\xf0\xa0\xa4\xe6\xac\xb5\xe1 \xc6\x87.\x9bOp\x97\x07\xd1\xa0\xf1\xdb\xb8ar\xa6\xbfEl\x81\x08:&\xf7\xb0\xf4\xfa\xaaI\xa1\xfc\x04\x1c|\xbe\xd0pU\x10=\x1aH}\xe1\xe7M!\x01|]\xab\xacs\xfd:\x1d\xab\xafg\xbb\xa5\x11\xae\xf3*;\xd4Q\xaf&\x7fYZ'\xe6\xbb\x99\xbc\xf1\x00\xd9\xa4\xee*\xa5T\x8c[\xaf\xce\xbeX\x112\xc7\x92\xc0v?\xd7G\x14\x91\xd8\x8dL\x9b\x1b\xecV\x91\x0e\xff\x00\xff^\x90\x86\xf2\xd3\xd4k"
DpqD1D+uBKQ3HAzv+lx1/LJ35+MZpmNq1Pzt3sOHA6mUJ3CptbATMnWUJPzJ84HPVileGOyBEyrI8PqMm3O9bnU3yXFTSRVhu4nO0Rue2nyCz+2aaojTzshV99UQINQatIKvTNH5+f3B2qzptRI/2ws36ueZNRSLl5FyyaAC8KCk5qy14SDGhy

Now decrypting it. In principle if we do it now in reverse we would get the same message.

In [20]:
# Splitting the string
split_name_1 = second_encrypted_base64_name[:344]
split_name_2 = second_encrypted_base64_name[344:]
print(split_name_1)
print(split_name_2)

# First decrypt
encrypted_name_1 = base64.b64decode(split_name_1)
print(encrypted_name_1)
decrypted_name_1 = decrypt_with_private_key(encrypted_name_1)
print(decrypted_name_1)

# Second decrypt
encrypted_name_2 = base64.b64decode(split_name_2)
print(encrypted_name_2)
decrypted_name_2 = decrypt_with_private_key(encrypted_name_2)
print(decrypted_name_2)

# Combining it
full_decrypted_name = decrypted_name_1+decrypted_name_2
print(full_decrypted_name)

# Perform first level decryption
encrypted_name = base64.b64decode(full_decrypted_name)
print(encrypted_name)
decrypted_name = decrypt_with_private_key(encrypted_name)
print(decrypted_name)


hgPxtPJ43qAOf8oPTwUEGq5s0rwuszblEilOAioClhJIyrjhexq+Y6pgHiJicERJsIKV41hbCqjJQX/48rOmOu/2SL9HBnN5Ld5uJAETxryN+YtgNCQ+HkIlVmukzadpVX2nc4k+Ce/inwp7fYeZk3nKo4yia7wU0BGaa21wTGZAk2MtetSa5nsNkWuxG/hXuFRpIXWXiSKBKuNawggHTPnWXqbcVNJOmfV+1kmkcXVUXgrT7EQnwhlAoGd9DvrtJztkfT85rOQiAdbI3e4oH01N6xZbJu66RYQGFA0Uj2AQPDkb84Rc7qjo9+DSHJJ6RX52Cd7K8lI6Cv5X3OKSaA==
Nt3b3DvHlivY8DiQ1pijyfM2ER7yAUp20Eq6D9BC/N72KHy+j/WgjW4LIClEjgK+LOf9VQffnfeWJXo0bh2YNlJVV1UDsBxkaroxLB3DSZL7zMW3YVtpkJp3vhFdunURKFbC2//hdfsqg202yVC4RecvrrM7VtehpGxKNtuV3+CqfrzwPwoWRQraQDrF2LakZwJBZilN56+IiPCfsC6t8T3xbwbj2wfaLYtWeef2U2iX/TCMKdmKStFA4vpXKI2TEGiyJWSlpqaP3gwLWehLB7BRbgB0Ruj0b0utN2kGClA7MGxU4CWbpBD2cGYa0xqQBfLV/MNQOEKl95Ht9/Jrgg==
b'\x86\x03\xf1\xb4\xf2x\xde\xa0\x0e\x7f\xca\x0fO\x05\x04\x1a\xael\xd2\xbc.\xb36\xe5\x12)N\x02*\x02\x96\x12H\xca\xb8\xe1{\x1a\xbec\xaa`\x1e"bpDI\xb0\x82\x95\xe3X[\n\xa8\xc9A\x7f\xf8\xf2\xb3\xa6:\xef\xf6H\xbfG\x06sy-\xden$\x01\x13\xc6\xbc\x8d\xf9\x8b`4$>\x1eB%Vk\xa4\xcd\xa7iU}\xa7s\x89>\t\xef\xe2\x9f\n{}\x87\x9

It seems we got the same message but we can check with the next step. Which seems it's the exact same string

In [21]:
print(name==decrypted_name)

True


## Conclusions

### Double encryption
In principle this could be something that enhances the security with having it be 2 encrypted parts but is it worth the extra implementation.   
I don't think so although it's fast you will have double the code for something that could be enough with the first encryption. Of course making it into methods will make it less padded but you need to keep in mind the length of things (base64 strings mostly).

So to keep it simple I do not recommend to encrypt twice as it brings:
- Redundancy: As long as you're using a strong encryption algorithm it's fine to encrypt it once.
- Weakness: Because of how it can be structured certain attacks could be done like meet-in-the-middle attacks.
- Performance: With longer strings it brings into question how long it will take or with multiple columns instead of only "name".
- Key management: Although I'm using 1 pair, if I used multiple it adds complexity to key management.

### Characters

If you can add it to a string it will give the correct character back. Doesn't matter if it's unicode or not.  
So, based on that you also have multilingual support so when this project will be translated to other languages without the latin alphabet.
